In [12]:
import pandas as pd
import os
import pandas as pd
import sys
sys.path.insert(0, '/home/jupyter/')
from hourly_pollution_prediction.process_and_join.bbox import * 
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
#for image inpainting
from skimage import data
from skimage.morphology import disk, binary_dilation
from skimage.restoration import inpaint
import cv2 as cv
import xesmf as xe
import geopandas as gpd
import rioxarray
from rioxarray.merge import merge_arrays
from pathlib import Path


In [18]:
df = pd.read_parquet('../../data/gridded_census_data/gridded_eif_pop_ageracesex_2024.parquet')
df = df[df['grid_lat'].astype(float)>=lat_min]
df = df[df['grid_lat'].astype(float)<=lat_max]
df = df[df['grid_lon'].astype(float)>=lon_min]
df = df[df['grid_lon'].astype(float)<=lon_max]

df['grid_lon']=df['grid_lon'].astype(float)
df['grid_lat']=df['grid_lat'].astype(float)

df = df.groupby(['grid_lon','grid_lat','race_ethnicity'])['n_noise'].sum().reset_index()

# Step 2: Pivot to get 3D shape (lat, lon, race)
df = df.pivot_table(
    index=["grid_lat", "grid_lon"],
    columns="race_ethnicity",
    values="n_noise",
    aggfunc="mean"  # or "sum" or "first" depending on how you want to handle duplicates
)

# Step 3: Convert to xarray
ds = df.to_xarray()
ds = ds.rename({var: var.replace("/", "_") for var in ds.data_vars})
ds.to_netcdf('../../data/gridded_census_data/ds_race.nc')

df2 = pd.read_parquet('../../data/gridded_census_data/gridded_eif_pop_raceincome_2024.parquet')
df2 = df2[df2['grid_lat'].astype(float)>=lat_min]
df2 = df2[df2['grid_lat'].astype(float)<=lat_max]
df2 = df2[df2['grid_lon'].astype(float)>=lon_min]
df2 = df2[df2['grid_lon'].astype(float)<=lon_max]

df2['grid_lon']=df2['grid_lon'].astype(float)
df2['grid_lat']=df2['grid_lat'].astype(float)

df2 = df2.groupby(['grid_lon','grid_lat','income_decile'])['n_noise'].sum().reset_index()

# Step 2: Pivot to get 3D shape (lat, lon, race)
df2 = df2.pivot_table(
    index=["grid_lat", "grid_lon"],
    columns="income_decile",
    values="n_noise",
    aggfunc="mean"  # or "sum" or "first" depending on how you want to handle duplicates
)

# Step 3: Convert to xarray
ds2 = df2.to_xarray()
ds2 = ds2.rename({var: str(var) for var in ds2.data_vars})
ds2.to_netcdf('../../data/gridded_census_data/ds_income.nc')


AttributeError: 'float' object has no attribute 'replace'

In [21]:
ds2

<xarray.Dataset> Size: 41MB
Dimensions:   (grid_lat: 480, grid_lon: 1060)
Coordinates:
  * grid_lat  (grid_lat) float64 4kB 28.61 28.61 28.62 ... 33.38 33.38 33.4
  * grid_lon  (grid_lon) float64 8kB -98.89 -98.89 -98.88 ... -88.31 -88.31
Data variables:
    1.0       (grid_lat, grid_lon) float64 4MB nan nan nan nan ... nan nan nan
    2.0       (grid_lat, grid_lon) float64 4MB nan nan nan nan ... nan 7.0 nan
    3.0       (grid_lat, grid_lon) float64 4MB nan nan nan nan ... nan 3.0 nan
    4.0       (grid_lat, grid_lon) float64 4MB nan nan nan nan ... nan nan nan
    5.0       (grid_lat, grid_lon) float64 4MB nan nan nan nan ... nan nan nan
    6.0       (grid_lat, grid_lon) float64 4MB nan nan nan nan ... nan nan nan
    7.0       (grid_lat, grid_lon) float64 4MB nan nan nan nan ... nan 1.0 nan
    8.0       (grid_lat, grid_lon) float64 4MB nan nan nan nan ... nan nan nan
    9.0       (grid_lat, grid_lon) float64 4MB nan nan nan nan ... 7.0 nan nan
    10.0      (grid_lat, grid_lon) float64 4MB nan nan nan nan ... nan 6.0 nan